In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com//oauth2/token"}


dbutils.fs.mount(
source = "abfss://e-commerce@ecommercdata.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/commerce",
extra_configs = configs)
  

True

In [0]:
%fs
ls "/mnt/commerce"


path,name,size,modificationTime
dbfs:/mnt/commerce/raw-data/,raw-data/,0,1726574848000
dbfs:/mnt/commerce/transformed-data/,transformed-data/,0,1726574862000


In [0]:
orders = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/commerce/raw-data/orders")
order_details = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/commerce/raw-data/orderdetails")
target = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/commerce/raw-data/target")

In [0]:
orders.printSchema()
order_details.printSchema()
target.printSchema()

root
 |-- Order ID: string (nullable = true)
 |-- Order Date: date (nullable = true)
 |-- CustomerName: string (nullable = true)
 |-- State: string (nullable = true)
 |-- City: string (nullable = true)

root
 |-- Order ID: string (nullable = true)
 |-- Amount: double (nullable = true)
 |-- Profit: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)

root
 |-- Month of Order Date: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Target: double (nullable = true)



In [0]:
# Drop rows where Order ID or Order Date is null
orders_clean = orders.dropna(subset=["Order ID", "Order Date"])

# Optionally, fill missing values in 'CustomerName', 'State', and 'City' with "Unknown"
orders_clean = orders_clean.fillna({"CustomerName": "Unknown", "State": "Unknown", "City": "Unknown"})

orders_clean.show()
orders_clean.count()
print(f"count of rows before cleaning : {orders.count()} , after cleaning : {orders_clean.count()}")


+--------+----------+------------+-----------------+------------------+
|Order ID|Order Date|CustomerName|            State|              City|
+--------+----------+------------+-----------------+------------------+
| B-25601|2018-04-01|      Bharat|          Gujarat|         Ahmedabad|
| B-25602|2018-04-01|       Pearl|      Maharashtra|              Pune|
| B-25603|2018-04-03|       Jahan|   Madhya Pradesh|            Bhopal|
| B-25604|2018-04-03|      Divsha|        Rajasthan|            Jaipur|
| B-25605|2018-04-05|     Kasheen|      West Bengal|           Kolkata|
| B-25606|2018-04-06|       Hazel|        Karnataka|         Bangalore|
| B-25607|2018-04-06|    Sonakshi|Jammu and Kashmir|           Kashmir|
| B-25608|2018-04-08|     Aarushi|       Tamil Nadu|           Chennai|
| B-25609|2018-04-09|      Jitesh|    Uttar Pradesh|           Lucknow|
| B-25610|2018-04-09|      Yogesh|            Bihar|             Patna|
| B-25611|2018-04-11|       Anita|          Kerala |Thiruvananth

In [0]:
# Drop rows where Order ID, Amount, or Profit is null
order_details_clean = order_details.dropna(subset=["Order ID", "Amount", "Profit"])

# Fill missing values in Category and Sub-Category with "Unknown"
order_details_clean = order_details_clean.fillna({"Category": "Unknown", "Sub-Category": "Unknown"})

order_details_clean.show()
print(f"count of rows before cleaning : {order_details.count()} , after cleaning : {order_details_clean.count()}")

+--------+------+-------+--------+-----------+----------------+
|Order ID|Amount| Profit|Quantity|   Category|    Sub-Category|
+--------+------+-------+--------+-----------+----------------+
| B-25601|1275.0|-1148.0|       7|  Furniture|       Bookcases|
| B-25601|  66.0|  -12.0|       5|   Clothing|           Stole|
| B-25601|   8.0|   -2.0|       3|   Clothing|     Hankerchief|
| B-25601|  80.0|  -56.0|       4|Electronics|Electronic Games|
| B-25602| 168.0| -111.0|       2|Electronics|          Phones|
| B-25602| 424.0| -272.0|       5|Electronics|          Phones|
| B-25602|2617.0| 1151.0|       4|Electronics|          Phones|
| B-25602| 561.0|  212.0|       3|   Clothing|           Saree|
| B-25602| 119.0|   -5.0|       8|   Clothing|           Saree|
| B-25603|1355.0|  -60.0|       5|   Clothing|        Trousers|
| B-25603|  24.0|  -30.0|       1|  Furniture|          Chairs|
| B-25603| 193.0| -166.0|       3|   Clothing|           Saree|
| B-25603| 180.0|    5.0|       3|   Clo

In [0]:
# Drop rows where Category or Month of Order Date is null
target_clean = target.dropna(subset=["Month of Order Date", "Category"])

# Fill missing values in 'Target' with 0 if nulls represent missing data
target_clean = target_clean.fillna({"Target": 0})

target_clean.show()
print(f"count of rows before cleaning : {target.count()} , after cleaning : {target_clean.count()}")


+-------------------+---------+-------+
|Month of Order Date| Category| Target|
+-------------------+---------+-------+
|             Apr-18|Furniture|10400.0|
|             May-18|Furniture|10500.0|
|             Jun-18|Furniture|10600.0|
|             Jul-18|Furniture|10800.0|
|             Aug-18|Furniture|10900.0|
|             Sep-18|Furniture|11000.0|
|             Oct-18|Furniture|11100.0|
|             Nov-18|Furniture|11300.0|
|             Dec-18|Furniture|11400.0|
|             Jan-19|Furniture|11500.0|
|             Feb-19|Furniture|11600.0|
|             Mar-19|Furniture|11800.0|
|             Apr-18| Clothing|12000.0|
|             May-18| Clothing|12000.0|
|             Jun-18| Clothing|12000.0|
|             Jul-18| Clothing|14000.0|
|             Aug-18| Clothing|14000.0|
|             Sep-18| Clothing|14000.0|
|             Oct-18| Clothing|16000.0|
|             Nov-18| Clothing|16000.0|
+-------------------+---------+-------+
only showing top 20 rows

count of rows 

In [0]:
orders_full = orders_clean.join(order_details_clean, on="Order ID", how="inner")

orders_full.show()

+--------+----------+------------+--------------+---------+------+-------+--------+-----------+----------------+
|Order ID|Order Date|CustomerName|         State|     City|Amount| Profit|Quantity|   Category|    Sub-Category|
+--------+----------+------------+--------------+---------+------+-------+--------+-----------+----------------+
| B-25601|2018-04-01|      Bharat|       Gujarat|Ahmedabad|1275.0|-1148.0|       7|  Furniture|       Bookcases|
| B-25601|2018-04-01|      Bharat|       Gujarat|Ahmedabad|  66.0|  -12.0|       5|   Clothing|           Stole|
| B-25601|2018-04-01|      Bharat|       Gujarat|Ahmedabad|   8.0|   -2.0|       3|   Clothing|     Hankerchief|
| B-25601|2018-04-01|      Bharat|       Gujarat|Ahmedabad|  80.0|  -56.0|       4|Electronics|Electronic Games|
| B-25602|2018-04-01|       Pearl|   Maharashtra|     Pune| 168.0| -111.0|       2|Electronics|          Phones|
| B-25602|2018-04-01|       Pearl|   Maharashtra|     Pune| 424.0| -272.0|       5|Electronics| 

In [0]:
from pyspark.sql.functions import date_format

# Create a new column 'Year-Month' from 'Order Date'
orders_full = orders_full.withColumn("Year-Month", date_format("Order Date", "yyyy-MM"))

orders_full.show()

+--------+----------+------------+--------------+---------+------+-------+--------+-----------+----------------+----------+
|Order ID|Order Date|CustomerName|         State|     City|Amount| Profit|Quantity|   Category|    Sub-Category|Year-Month|
+--------+----------+------------+--------------+---------+------+-------+--------+-----------+----------------+----------+
| B-25601|2018-04-01|      Bharat|       Gujarat|Ahmedabad|1275.0|-1148.0|       7|  Furniture|       Bookcases|   2018-04|
| B-25601|2018-04-01|      Bharat|       Gujarat|Ahmedabad|  66.0|  -12.0|       5|   Clothing|           Stole|   2018-04|
| B-25601|2018-04-01|      Bharat|       Gujarat|Ahmedabad|   8.0|   -2.0|       3|   Clothing|     Hankerchief|   2018-04|
| B-25601|2018-04-01|      Bharat|       Gujarat|Ahmedabad|  80.0|  -56.0|       4|Electronics|Electronic Games|   2018-04|
| B-25602|2018-04-01|       Pearl|   Maharashtra|     Pune| 168.0| -111.0|       2|Electronics|          Phones|   2018-04|
| B-2560

In [0]:
from pyspark.sql import functions as F

# Group by Year-Month and Category to get total sales and profit
monthly_sales_profit = orders_full.groupBy("Year-Month", "Category").agg(
                                                                        F.sum("Amount").alias("Total Sales"),
                                                                        F.sum("Profit").alias("Total Profit")
                                                                    )

monthly_sales_profit.show()

+----------+-----------+-----------+------------+
|Year-Month|   Category|Total Sales|Total Profit|
+----------+-----------+-----------+------------+
|   2019-03|Electronics|    20860.0|      3430.0|
|   2018-12|Electronics|    18560.0|      3196.0|
|   2018-09|   Clothing|    10717.0|     -1585.0|
|   2018-07|   Clothing|     2981.0|       -48.0|
|   2018-10|   Clothing|    11488.0|      2148.0|
|   2018-06|  Furniture|     5532.0|      -856.0|
|   2019-02|   Clothing|     9569.0|      1822.0|
|   2018-05|   Clothing|     9518.0|      -267.0|
|   2018-05|  Furniture|     6220.0|      -794.0|
|   2018-11|  Furniture|    15165.0|      3945.0|
|   2018-09|Electronics|     7207.0|      -910.0|
|   2019-03|  Furniture|    16659.0|      1587.0|
|   2018-04|Electronics|    11127.0|      -351.0|
|   2019-02|Electronics|    12593.0|      1927.0|
|   2019-01|Electronics|    26716.0|      4785.0|
|   2018-04|  Furniture|     8121.0|     -3425.0|
|   2019-03|   Clothing|    21418.0|      5060.0|


In [0]:
orders_clean.write.mode("overwrite").csv("/mnt/commerce/transformed-data/orders", header=True)
target_clean.write.mode("overwrite").csv("/mnt/commerce/transformed-data/target", header=True)
order_details_clean.write.mode("overwrite").csv("/mnt/commerce/transformed-data/order_details", header=True)
orders_full.write.mode("overwrite").csv("/mnt/commerce/transformed-data/orders_full", header=True)
monthly_sales_profit.write.mode("overwrite").csv("/mnt/commerce/transformed-data/monthly_sales_profit", header=True)



